#Cassava Leaf Disease Classification Project#

##Part 1: Import Packages and the Dataset##

####Import Packages###

In [13]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyClassifier
from google.colab import files
import pandas as pd
import io
from keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from google.colab import drive
import json
from tensorflow import keras
import tensorflow as tf

In [14]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
import random
import tensorflow as tf
import cv2
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors

In [15]:
from sklearn.model_selection import train_test_split
import re
from functools import partial

In [16]:
import keras
from keras.layers import Dense, Dropout, Input, MaxPooling2D, ZeroPadding2D, Conv2D, Flatten, BatchNormalization
from keras.models import Sequential, Model
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from keras.utils import to_categorical
from tensorflow.keras import regularizers

from tensorflow.keras.layers import MaxPool2D, AveragePooling2D, GlobalAveragePooling2D
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

from zipfile import ZipFile
import time
from datetime import timedelta
from io import BytesIO

# Image manipulation.
import PIL.Image

import pickle
import os

import random

###Mount the Drive###

In [17]:
import zipfile
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [18]:
f = open('/content/gdrive/Shareddrives/2040 (unofficial)/my_data/data/label_num_to_disease_map.json')
label_name = json.load(f)
label_name

{'0': 'Cassava Bacterial Blight (CBB)',
 '1': 'Cassava Brown Streak Disease (CBSD)',
 '2': 'Cassava Green Mottle (CGM)',
 '3': 'Cassava Mosaic Disease (CMD)',
 '4': 'Healthy'}

###Load the tfrecords###

***Define Some Important Variables***
* Batch_Size: 128
* Labels: 5 different labels in total 
* tf.data.AUTOTUNE: promp the tf.data runtime to tune the value dynamically at runtime. 
* train_path: the path of training dataset directory
* test_path: the path of testing dataset directory
* Image_Size: 512*512

In [19]:
Batch_Size = 128
labels = list(label_name.keys())
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_path = '/content/gdrive/Shareddrives/2040 (unofficial)/my_data/data/train_tfrecords/ld_train*.tfrec'
test_path = '/content/gdrive/Shareddrives/2040 (unofficial)/my_data/data/test_tfrecords/ld_test*.tfrec'
Image_Size_Original = [512, 512]
image_size = 340

In [20]:
labels

['0', '1', '2', '3', '4']

***Preprocess the Image***
* Decode a JPEG-encoded image to a uint8 tensor.
* Divide by 255 to get 0-1 representation 

In [21]:
def preprocess_img(img):
  img_decode = tf.image.decode_jpeg(img, channels = 3)
  img_255 = tf.cast(img_decode, tf.float32)/255.0
  img_res = tf.reshape(img_255, [*Image_Size_Original,3])
  img_resize = tf.image.resize(img_res, [image_size,image_size])
  return img_resize

***Read the tfrecord***

In [22]:
##
def read_tfrecord(example, labeled):
  tfrecord_format = {"image":tf.io.FixedLenFeature([], tf.string),
                     "target":tf.io.FixedLenFeature([], tf.int64),
                     } if labeled else {
                     "image": tf.io.FixedLenFeature([], tf.string),
                     "image_name": tf.io.FixedLenFeature([], tf.string)
                     }
  example = tf.io.parse_single_example(example, tfrecord_format)
  
  img_res = preprocess_img(example['image'])
  
  if labeled:
    label = tf.cast(example['target'], tf.int32)
    return img_res, label
  else: 
    id = example['image_name']
    return img_res, id

In [23]:
def read_tfrecord(example, labeled):
    tfrecord_format = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.int64)
    } if labeled else {
        "image": tf.io.FixedLenFeature([], tf.string),
        "image_name": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = preprocess_img(example['image'])
    if labeled:
        label = tf.cast(example['target'], tf.int32)
        return image, label
    idnum = example['image_name']
    return image, idnum

***Load the dataset***

In [24]:
##
def load_dataset(filenames, labeled=True, ordered=False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTOTUNE) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(partial(read_tfrecord, labeled=labeled), num_parallel_calls=AUTOTUNE)
    return dataset

###Splitting Strategy###

In [25]:
df_training, df_validation = train_test_split(tf.io.gfile.glob(train_path),
                                              test_size=0.2, random_state=42)
df_testing = tf.io.gfile.glob(test_path)

In [26]:
len(df_training), len(df_validation), len(df_testing)

(12, 4, 1)

In [27]:
df_training, df_validation, df_testing

(['/content/gdrive/Shareddrives/2040 (unofficial)/my_data/data/train_tfrecords/ld_train13-1338.tfrec',
  '/content/gdrive/Shareddrives/2040 (unofficial)/my_data/data/train_tfrecords/ld_train11-1338.tfrec',
  '/content/gdrive/Shareddrives/2040 (unofficial)/my_data/data/train_tfrecords/ld_train08-1338.tfrec',
  '/content/gdrive/Shareddrives/2040 (unofficial)/my_data/data/train_tfrecords/ld_train09-1338.tfrec',
  '/content/gdrive/Shareddrives/2040 (unofficial)/my_data/data/train_tfrecords/ld_train02-1338.tfrec',
  '/content/gdrive/Shareddrives/2040 (unofficial)/my_data/data/train_tfrecords/ld_train15-1327.tfrec',
  '/content/gdrive/Shareddrives/2040 (unofficial)/my_data/data/train_tfrecords/ld_train04-1338.tfrec',
  '/content/gdrive/Shareddrives/2040 (unofficial)/my_data/data/train_tfrecords/ld_train07-1338.tfrec',
  '/content/gdrive/Shareddrives/2040 (unofficial)/my_data/data/train_tfrecords/ld_train10-1338.tfrec',
  '/content/gdrive/Shareddrives/2040 (unofficial)/my_data/data/train_tfre

###Image Augmentation###

In [28]:
def train_augmentation(image, label):
#image_augmentation 
  image = tf.image.random_flip_left_right(image)
  image = tf.image.random_flip_up_down(image)
 # image = tf.image.random_hue(image, 0.2)
  image = tf.image.random_crop(image, [image_size, image_size, 3])
 # image = tf.image.random_brightness(image, max_delta=0.5)
 # image = tf.image.random_saturation(image, lower = 5, upper = 10)
 # image = tf.image.random_contrast(image, 0.2, 0.5)
 # image = tf.image.random_jpeg_quality(image, 75, 95)
  return image, label

###Load the dataset###

* repeat: repeats this dataset so each original value is seen infinitely. 
* shuffle: randomly shuffles the elements of this dataet. 
* prefetch: allows later elements to be prepared while the current element is being processed.

In [29]:
def get_training_dataset():
    df_train = load_dataset(df_training, labeled=True)  
    df_train = df_train.map(train_augmentation, num_parallel_calls=AUTOTUNE)  
    df_train = df_train.repeat()
    df_train = df_train.shuffle(2048)
    df_train = df_train.batch(Batch_Size)
    df_train = df_train.prefetch(AUTOTUNE) # allows later elements to be prepared while the current element is being processed.
    return df_train

In [30]:
def get_validation_dataset(ordered=False):
    df_valid = load_dataset(df_validation, labeled=True, ordered=ordered) 
    df_valid = df_valid.batch(Batch_Size)
    df_valid = df_valid.cache()
    df_valid = df_valid.prefetch(AUTOTUNE)
    return df_valid

In [31]:
def get_test_dataset(ordered=False):
    df_test = load_dataset(df_testing, labeled=False, ordered=ordered)
    df_test = df_test.batch(Batch_Size)
    df_test = df_test.prefetch(AUTOTUNE)
    return df_test

In [32]:
def counting_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

In [33]:
num_training = counting_items(df_training)
num_valid = counting_items(df_validation)
num_test = counting_items(df_testing)

print('This Dataset: {} training images, {} validation images, {} (unlabeled) test images'.format(
    num_training, num_valid, num_test))

This Dataset: 16045 training images, 5352 validation images, 1 (unlabeled) test images


In [34]:
df_train = get_training_dataset()
df_valid = get_validation_dataset()
df_test = get_test_dataset()

In [35]:
df_train, df_valid, df_test

(<PrefetchDataset shapes: ((None, 340, 340, 3), (None,)), types: (tf.float32, tf.int32)>,
 <PrefetchDataset shapes: ((None, 340, 340, 3), (None,)), types: (tf.float32, tf.int32)>,
 <PrefetchDataset shapes: ((None, 340, 340, 3), (None,)), types: (tf.float32, tf.string)>)

In [36]:
print("Training dataset:")
for image, label in df_train.take(3):
    print(image.numpy().shape, label.numpy().shape)
    print("Training dataset labels:", label.numpy())
print("Validation dataset:")
for image, label in df_valid.take(3):
    print(image.numpy().shape, label.numpy().shape)
    print("Validation dataset labels:", label.numpy())
print("Test dataset:")
for image, idnum in df_test.take(3):
    print(image.numpy().shape, idnum.numpy().shape)
    print("Test data IDs:", idnum.numpy().astype('U')) 


Training dataset:
(128, 340, 340, 3) (128,)
Training dataset labels: [3 3 3 0 3 0 3 4 3 3 3 3 2 3 0 3 1 3 2 3 2 2 3 3 3 0 0 1 3 3 3 3 3 4 3 4 3
 3 0 4 3 3 3 3 3 3 3 4 4 3 3 3 3 3 1 4 3 4 2 2 3 3 3 3 2 3 3 3 3 1 2 4 3 3
 3 4 1 3 2 3 3 3 3 4 3 4 3 3 3 3 3 3 3 3 3 3 2 3 4 3 3 3 1 4 3 1 3 3 3 4 3
 1 3 3 3 3 3 3 3 1 2 3 3 3 3 3 4 3]
(128, 340, 340, 3) (128,)
Training dataset labels: [3 3 1 0 2 3 3 3 1 3 2 1 3 3 3 4 3 3 0 3 3 0 4 4 3 4 1 3 3 3 3 3 0 3 1 2 3
 3 4 4 3 3 3 1 3 4 3 3 0 3 3 3 3 3 4 3 3 4 3 3 3 3 2 4 3 1 3 4 3 0 3 4 3 3
 3 3 3 0 1 3 2 3 3 3 4 3 2 3 1 4 3 3 3 3 3 4 3 4 4 3 3 3 3 3 1 3 1 2 3 0 3
 3 3 3 3 3 1 3 4 3 3 3 4 3 3 1 3 3]
(128, 340, 340, 3) (128,)
Training dataset labels: [3 3 2 3 3 1 2 3 3 0 0 3 4 3 3 2 2 3 3 3 3 3 2 3 2 3 3 3 1 4 2 3 3 1 3 3 3
 3 3 3 4 3 3 4 1 3 3 3 1 3 4 3 1 3 1 3 3 3 1 2 3 3 4 3 3 2 2 3 3 3 2 2 2 3
 3 4 1 2 4 3 3 3 3 3 2 1 0 0 3 2 3 2 2 4 2 3 3 3 3 3 3 3 3 4 3 3 3 3 3 2 3
 3 3 3 0 0 3 3 3 2 3 3 3 3 3 2 1 3]
Validation dataset:
(128, 340, 340, 3) (128,)


##Part2: Model Selection

In [42]:
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import AveragePooling2D, GlobalMaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras import callbacks
from keras.applications import *
from keras.callbacks import LearningRateScheduler,ReduceLROnPlateau, ModelCheckpoint


### Resnet101V2

In [43]:
steps_per_epoch = num_training// Batch_Size
valid_steps = num_valid // Batch_Size

In [44]:
early_stopping1 = callbacks.EarlyStopping(patience= 10, restore_best_weights=True)

#### Define model with compile

In [46]:
def make_model(opt, reg_rate):
  base_resnet = tf.keras.applications.ResNet101V2(include_top = False, 
                        input_shape=(image_size, image_size, 3),
                        weights="imagenet", 
                        classifier_activation="softmax")
  for layer in base_resnet.layers:
    layer.trainable = False

  head = base_resnet.output

  head = GlobalMaxPooling2D()(head)
  head = Flatten()(head)
  head = Dense(512, kernel_initializer = 'he_normal', 
              kernel_regularizer=l1(reg_rate))(head)
  head = BatchNormalization()(head)
  head = Activation('relu')(head)
  head = Dropout(rate=0.5)(head)

  # head = Dense(64)(head)
  # head = BatchNormalization()(head)
  # #head = Activation('relu')(head)
  # head = LeakyReLU()(head)
  head = Dense(5, activation ='softmax')(head)

  model = Model(inputs = base_resnet.input, outputs =head)

  model.compile(
    optimizer = opt,
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)
  return model

In [47]:
opt = keras.optimizers.Adam(0.0001) 
model1 = make_model(opt, 0.01)

171319296/171317808 [==============================] - 3s 0us/step


In [48]:
history1 = model1.fit(df_train, validation_data= df_valid, steps_per_epoch=steps_per_epoch, 
                shuffle= True, epochs=100, callbacks=[early_stopping1])

Epoch 1/100
125/125 [==============================] - 155s 1s/step - loss: 241.4902 - accuracy: 0.4794 - val_loss: 163.2515 - val_accuracy: 0.6287
Epoch 2/100
125/125 [==============================] - 119s 952ms/step - loss: 142.7725 - accuracy: 0.5986 - val_loss: 89.3295 - val_accuracy: 0.6590
Epoch 3/100
125/125 [==============================] - 119s 953ms/step - loss: 76.3388 - accuracy: 0.6251 - val_loss: 43.5324 - val_accuracy: 0.6839
Epoch 4/100
125/125 [==============================] - 119s 952ms/step - loss: 36.2133 - accuracy: 0.6623 - val_loss: 18.5870 - val_accuracy: 0.6958
Epoch 5/100
125/125 [==============================] - 119s 954ms/step - loss: 15.1582 - accuracy: 0.6666 - val_loss: 7.4078 - val_accuracy: 0.7081
Epoch 6/100
125/125 [==============================] - 119s 953ms/step - loss: 6.2572 - accuracy: 0.6804 - val_loss: 3.9420 - val_accuracy: 0.6889
Epoch 7/100
125/125 [==============================] - 119s 952ms/step - loss: 3.7376 - accuracy: 0.6808 - va

KeyboardInterrupt: ignored

#### Adam Optimizer

Frozen  last 5 layers (last block), change batch size to 32

In [ ]:
batch_size = 32
steps_per_epoch = num_training// batch_size
valid_steps = num_valid // batch_size

In [ ]:
base_resnet = tf.keras.applications.ResNet101V2(include_top = False, 
                        input_shape=(image_size, image_size, 3),
                        weights="imagenet")
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001, cooldown=1)

for layer in base_resnet.layers[:-5]:
  layer.trainable = False

head = base_resnet.output

head = GlobalMaxPooling2D()(head)
head = Flatten()(head)

head = Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l1_l2(l2=0.001))(head)
head = BatchNormalization()(head)

head = Dense(64, activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l1_l2(l2=0.001))(head)
head = BatchNormalization()(head)

head = Dense(5, activation ='softmax')(head)

model2 = Model(inputs = base_resnet.input, outputs =head)

model2.compile(
  optimizer = opt,
  loss=keras.losses.SparseCategoricalCrossentropy(),
  metrics=['accuracy']
)


opt = keras.optimizers.Adam(0.0001) 

history2 = model2.fit(df_train, validation_data= df_valid, steps_per_epoch=steps_per_epoch, 
                shuffle= True, epochs=100, callbacks=[early_stopping1])

Epoch 1/100
501/501 [==============================] - 238s 428ms/step - loss: 33.8986 - accuracy: 0.4078 - val_loss: 23.3932 - val_accuracy: 0.5133
Epoch 2/100
501/501 [==============================] - 213s 425ms/step - loss: 20.2405 - accuracy: 0.6290 - val_loss: 13.3378 - val_accuracy: 0.5729
Epoch 3/100
501/501 [==============================] - 212s 424ms/step - loss: 11.2917 - accuracy: 0.6569 - val_loss: 7.3689 - val_accuracy: 0.5854
Epoch 4/100
255/501 [==============>...............] - ETA: 1:35 - loss: 6.5958 - accuracy: 0.6675

KeyboardInterrupt: ignored

In [ ]:
## !git clone https://github.com/christianversloot/CLR.git


fatal: destination path 'CLR' already exists and is not an empty directory.


#### Cyclic learning rate (CLR) trial

Cyclic learning rate (CLR) with batch size 128 and adam optimizer learning rate as 0.0001. CLR helps training process escape from local minimum or converge faster when starts new training plateau.

In [ ]:
batch_size = 128
steps_per_epoch = num_training// batch_size
valid_steps = num_valid // batch_size 

In [ ]:
from CLR.clr_callback import CyclicLR

# Set CLR options
clr_step_size = int(4 * (num_training/batch_size))
base_lr = 1e-4
max_lr = 1e-2
mode='triangular'
no_epochs = 50

clr = CyclicLR(base_lr = base_lr, max_lr = max_lr, step_size = clr_step_size, mode = mode)


In [ ]:
base_resnet = tf.keras.applications.ResNet101V2(include_top = False, 
                        input_shape=(image_size, image_size, 3),
                        weights="imagenet")
for layer in base_resnet.layers[:-10]:
  layer.trainable = False

head = base_resnet.output

head = GlobalMaxPooling2D()(head)
head = Flatten()(head)


head = Dense(64, activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l1_l2(l2=0.01))(head)
head = BatchNormalization()(head)

head = Dense(5, activation ='softmax')(head)

model = Model(inputs = base_resnet.input, outputs =head)

model.compile(
  optimizer = opt,
  loss=keras.losses.SparseCategoricalCrossentropy(),
  metrics=['accuracy']
)

opt = keras.optimizers.Adam(0.0001) 

history3 = model3.fit(df_train, validation_data= df_valid, steps_per_epoch=steps_per_epoch, 
                shuffle= True, epochs=100, callbacks=[early_stopping1, clr])

Epoch 1/50
125/125 [==============================] - 75s 503ms/step - loss: 36.9438 - accuracy: 0.3186 - val_loss: 34.3223 - val_accuracy: 0.3668
Epoch 2/50
125/125 [==============================] - 60s 484ms/step - loss: 32.7799 - accuracy: 0.4941 - val_loss: 30.7869 - val_accuracy: 0.3552
Epoch 3/50
125/125 [==============================] - 60s 478ms/step - loss: 29.0696 - accuracy: 0.5691 - val_loss: 26.9652 - val_accuracy: 0.4524
Epoch 4/50
125/125 [==============================] - 58s 467ms/step - loss: 25.6573 - accuracy: 0.6011 - val_loss: 23.7616 - val_accuracy: 0.4737
Epoch 5/50
125/125 [==============================] - 56s 451ms/step - loss: 22.5502 - accuracy: 0.6249 - val_loss: 20.9080 - val_accuracy: 0.4688
Epoch 6/50
125/125 [==============================] - 56s 445ms/step - loss: 19.7456 - accuracy: 0.6337 - val_loss: 18.0899 - val_accuracy: 0.5575
Epoch 7/50
125/125 [==============================] - 56s 449ms/step - loss: 17.2152 - accuracy: 0.6473 - val_loss: 15

#### Reduce learning rate schedule

In [ ]:
batch_size = 32
steps_per_epoch = num_training// batch_size
valid_steps = num_valid // batch_size

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.00001, cooldown=1)

In [ ]:
base_resnet = tf.keras.applications.ResNet101V2(include_top = False, 
                        input_shape=(image_size, image_size, 3),
                        weights="imagenet")
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=5, min_lr=0.0001, cooldown=1)

for layer in base_resnet.layers[:-5]:
  layer.trainable = False

head = base_resnet.output

head = GlobalMaxPooling2D()(head)
head = Flatten()(head)

# head = Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01))(head)
# head = BatchNormalization()(head)

head = Dense(64, activation='relu', kernel_initializer='he_normal', kernel_regularizer=regularizers.l1_l2(l2=0.01))(head)
head = BatchNormalization()(head)

head = Dense(5, activation ='softmax')(head)

model = Model(inputs = base_resnet.input, outputs =head)

model.compile(
  optimizer = opt,
  loss=keras.losses.SparseCategoricalCrossentropy(),
  metrics=['accuracy']
)


opt = keras.optimizers.Adam(0.001) 

history1 = model.fit(df_train, validation_data= df_valid, steps_per_epoch=steps_per_epoch, validation_steps = valid_steps,
                shuffle= True, epochs=100, callbacks=[early_stopping1, reduce_lr])

NameError: ignored

#### Learning rate decay with another model

In [ ]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)

In [ ]:
opt = optimizer
model = make_model1(opt, 0.01)

In [ ]:
model.fit(df_train, validation_data= df_valid, steps_per_epoch=steps_per_epoch, validation_steps =valid_steps, 
                 shuffle= True, epochs=50, callbacks=[early_stopping1])

Epoch 1/50
50/50 [==============================] - 52s 626ms/step - loss: 94.8769 - accuracy: 0.4154 - val_loss: 90.0078 - val_accuracy: 0.4679
Epoch 2/50
22/50 [============>.................] - ETA: 11s - loss: 89.0968 - accuracy: 0.6190

KeyboardInterrupt: ignored

### Xception

In [ ]:
Batzh_Size = 64
steps_per_epoch = num_training// Batch_Size
valid_steps = num_valid // Batch_Size

In [ ]:
checkpoint_cb = ModelCheckpoint("Cassava_best_model.h5",
                  save_best_only=True,
                  monitor = 'val_loss',
                  mode='min')

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2,
                    patience = 10, min_lr = 1e-6,
                    mode = 'min', verbose = 1)

base_resnet = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_shape= (image_size, image_size, 3))

for layer in base_resnet.layers[40:]:
  layer.trainable = True
  
for layer in base_resnet.layers:
  if layer.name.endswith('_bn'):
    layer.trainable = False
head = base_resnet.output

head = GlobalMaxPooling2D()(head)
head = Flatten()(head)

head = Dense(64, activation = 'relu', bias_regularizer=tf.keras.regularizers.L1L2(l1=0.01, l2=0.001))(head)
head = BatchNormalization()(head)
head = Dropout(0.5)(head)

head = Dense(5, activation ='softmax')(head)

model = Model(inputs = base_resnet.input, outputs =head)
opt = keras.optimizers.Adam(0.001) 

model.compile(
  optimizer = opt,
  loss=keras.losses.SparseCategoricalCrossentropy(from_logits = False),
  metrics=['accuracy']
)

h = model.fit(df_train, validation_data= df_valid, steps_per_epoch=steps_per_epoch, validation_steps=valid_steps,
                    #validation_steps=valid_steps,
                shuffle= True, epochs=300, callbacks=[checkpoint_cb])

83689472/83683744 [==============================] - 1s 0us/step
Epoch 1/300


ResourceExhaustedError: ignored

### Resnet101V2 with 3 dense layers

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2,
                    patience = 2, min_lr = 1e-6,
                    mode = 'min', verbose = 1)
base_resnet = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3),
)

for layer in base_resnet.layers[:-5]:
  layer.trainable = False
#  print(layer, layer.trainable)

for layer in base_resnet.layers:
  if layer.name.endswith('_bn'):
    layer.trainable = False

head = base_resnet.output

head = GlobalAveragePooling2D()(head)

head = Flatten()(head)

head = Dense(128, kernel_initializer='he_normal', bias_regularizer=tf.keras.regularizers.L1L2(l1=0.01, l2=0.001))(head)
head = Activation('relu')(head)
head = BatchNormalization()(head)
head = Dropout(0.5)(head)

head = Dense(64, kernel_initializer='he_normal', bias_regularizer=tf.keras.regularizers.L1L2(l1=0.01, l2=0.001))(head)
head = Activation('relu')(head)
head = BatchNormalization()(head)
head = Dropout(0.3)(head)

head = Dense(5, activation ='softmax')(head)

model = Model(inputs = base_resnet.input, outputs =head)

opt = keras.optimizers.Adam(0.01, beta_1=0.9,  beta_2=0.999) 

model.compile(
  optimizer = opt,
  loss=keras.losses.SparseCategoricalCrossentropy(),
  metrics=['accuracy']
)

history = model.fit(df_train, validation_data= df_valid, steps_per_epoch=steps_per_epoch, 
                shuffle= True, epochs=100, callbacks=[reduce_lr])

Epoch 1/100
1002/1002 [==============================] - 428s 419ms/step - loss: 6.0164 - accuracy: 0.6270 - val_loss: 2.7592 - val_accuracy: 0.6454
Epoch 2/100
1002/1002 [==============================] - 408s 407ms/step - loss: 2.9979 - accuracy: 0.6667 - val_loss: 4.0840 - val_accuracy: 0.4144
Epoch 3/100
1002/1002 [==============================] - 408s 407ms/step - loss: 2.9172 - accuracy: 0.6739 - val_loss: 3.3950 - val_accuracy: 0.4475

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
Epoch 4/100
1002/1002 [==============================] - 407s 407ms/step - loss: 1.3375 - accuracy: 0.6875 - val_loss: 2.2724 - val_accuracy: 0.3602
Epoch 5/100
1002/1002 [==============================] - 404s 404ms/step - loss: 1.2006 - accuracy: 0.6905 - val_loss: 1.3191 - val_accuracy: 0.6512
Epoch 6/100
1002/1002 [==============================] - 400s 399ms/step - loss: 1.2084 - accuracy: 0.6917 - val_loss: 1.2566 - val_accuracy: 0.6691
Epoch 7/100
1002/1002 [==

KeyboardInterrupt: ignored

### Resnet101V2 with maxpooling after pre-trained

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.00001)

base_resnet = tf.keras.applications.ResNet101V2(include_top = False, 
                        input_shape=(image_size, image_size, 3),
                        weights="imagenet")

for layer in base_resnet.layers[:-5]:
  layer.trainable = False
#  print(layer, layer.trainable)

for layer in base_resnet.layers:
  if layer.name.endswith('_bn'):
    layer.trainable = False

head = base_resnet.output

head = GlobalMaxPooling2D()(head)
head = Flatten()(head)

head = Dense(64, kernel_initializer='he_normal', kernel_regularizer=regularizers.l1_l2(l2=0.001))(head)
head = Activation('relu')(head)
head = BatchNormalization()(head)
head = Dropout(0.5)(head)

head = Dense(5, activation ='softmax')(head)

model = Model(inputs = base_resnet.input, outputs =head)

opt = keras.optimizers.Adam(0.0001, beta_1=0.9,  beta_2=0.999) 

model.compile(
  optimizer = opt,
  loss=keras.losses.SparseCategoricalCrossentropy(),
  metrics=['accuracy']
)

history = model.fit(df_train, validation_data= df_valid, steps_per_epoch=steps_per_epoch, 
                shuffle= True, epochs=100, callbacks=[reduce_lr, model_checkpoint_callback])

NameError: ignored

In [ ]:
%tensorboard --logdir logs/data2040_midterm_project

2021-03-16 20:40:21.699072: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

logs

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=373649185512-8v619h5kft38l4456nm2dj4ubeqsrvh6.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scop